# **Computational Theory Assessment (2024/2025)**
Ronan Francis (G00403092)
Computational Theory

In [69]:
import math
import string
import random
import struct

## **Table of Contents**
1. [Task 1: Binary Representations](#task-1-binary-representations)
2. [Task 2: Hash Functions](#task-2-hash-functions)
3. [Task 3: SHA256 Padding](#task-3-sha256-padding)
4. [Task 4: Prime Numbers](#task-4-prime-numbers)
5. [Task 5: Roots](#task-5-roots)
6. [Task 6: Proof of Work](#task-6-proof-of-work)
7. [Task 7: Turing Machines](#task-7-turing-machines)
8. [Task 8: Computational Complexity](#task-8-computational-complexity)

---

# **Task 1: Binary Representations**

## Introduction 
In this task, we focus on implementing several core functions that manipulate binary representations of 32-bit unsigned integers. These operations are crucial in many fields, including cryptography (like SHA-256), data encoding, and low-level programming.

### Circular Shifts

Circular shifts (or bit rotations) involve shifting the bits of a number to the left or right, with the bits that "fall off" one end reappearing on the opposite end. This operation is distinct from logical shifts, where bits shifted off one end are simply discarded and zeros are introduced on the other end. Circular shifts maintain the bit-length and the overall "weight" of the data.

For example, when you perform a left circular shift (`rotl`) on a 32-bit integer:
- The bits are moved to the left by a specified number of positions.
- The bits that move past the most significant bit are wrapped around to the least significant bit positions.

Similarly, a right circular shift (`rotr`) moves bits to the right with wrapping from the rightmost bits to the leftmost positions.

For more detailed information, see the [Circular Shift Wikipedia article](https://en.wikipedia.org/wiki/Circular_shift).

### Bitwise Operators

Python provides several bitwise operators that allow for efficient manipulation of integers at the binary level:

- **AND (`&`):** Compares each bit of two integers and returns `1` only if both bits are `1`.
- **OR (`|`):** Compares each bit of two integers and returns `1` if at least one of the bits is `1`.
- **XOR (`^`):** Returns `1` for each bit position where the corresponding bits of the two operands are different.
- **NOT (`~`):** Inverts the bits of the operand.
- **Left Shift (`<<`):** Shifts the bits of a number to the left by a specified number of positions.
- **Right Shift (`>>`):** Shifts the bits of a number to the right by a specified number of positions.

These operators form the foundation for our binary manipulation functions. Detailed explanations and examples can be found at [GeeksforGeeks Bitwise Operators](https://www.geeksforgeeks.org/python-bitwise-operators/).

In summary, Task 1 involves implementing critical binary operations using circular shifts and bitwise operators. These functions serve as building blocks for more complex algorithms like SHA-256 and are widely applicable in many areas of computer science. By understanding and implementing `rotl`, `rotr`, `ch`, and `maj`, you not only gain proficiency in bitwise manipulation but also lay the groundwork for further exploration into cryptographic and low-level programming concepts.

For further reading and a deeper understanding, explore the [Circular Shift](https://en.wikipedia.org/wiki/Circular_shift) and [Bitwise Operators](https://www.geeksforgeeks.org/python-bitwise-operators/) articles.


## 1. [rotl(x, n=1)](https://en.cppreference.com/w/cpp/numeric/rotl)
   - **Purpose:** Rotates the bits of a 32-bit unsigned integer `x` to the left by `n` positions.
   - **Method:** Combine a left shift and a right shift using bitwise OR, ensuring the wrapped-around bits are correctly placed.
   - **Considerations:** Use a bitmask (`0xFFFFFFFF`) to maintain 32-bit constraints, and handle cases where `n >= 32` by reducing `n` modulo 32.

In [70]:
def rotl(x, n=1):
    # How many bits we need to represent x (at least 1)
    width = x.bit_length() or 1
    mask = (1 << width) - 1 # For example if width is 7, mask is 0b1111111

    # Make sure n is in the range [0, width]
    n %= width

    # Do the rotation:
    # - shift x to the left by n bits
    # - bitwise AND with the mask to keep only the width least significant bits
    # - bitwise OR with the bits that were "shifted out" to the left
    return ((x << n) & mask) | (x >> (width - n))

# Example
x = 0b1010101 # 85
rotated = rotl(x) # rotate left by 1

# Print the binary representation of x and the rotated value, with leading zeros
width = x.bit_length() or 1
print("Original:", format(x, '0{}b'.format(width)))
print("Rotated: ", format(rotated, '0{}b'.format(width)))


Original: 1010101
Rotated:  0101011


## 2. [rotr(x, n=1)](https://en.cppreference.com/w/cpp/numeric/rotr)
   - **Purpose:** Rotates the bits of a 32-bit unsigned integer `x` to the right by `n` positions.
   - **Method:** Similar to `rotl`, but with right and left shifts interchanged.
   - **Considerations:** Maintain 32-bit integrity using a bitmask and modulo operations on `n`.

In [71]:
def rotr(x, n=1):
    width = x.bit_length() or 1 # How many bits we need to represent x (at least 1)
    mask = (1 << width) - 1  # For example if width is 7, mask is 0b1111111
    x &= mask # Make sure x is in the range [0, 2**width)
    n %= width # Make sure n is in the range [0, width)
    return (x >> n) | ((x << (width - n)) & mask) # Do the rotation

# Example
x = 0b0101010 # 42
rotated = rotr(x) # rotate left by 1

# Print the binary representation of x and the rotated value, with leading zeros
width = x.bit_length() or 1
print("Original:", format(x, '0{}b'.format(width)))
print("Rotated: ", format(rotated, '0{}b'.format(width)))

Original: 101010
Rotated:  010101


## 3. [ch(x, y, z)](https://crypto.stackexchange.com/questions/5358/what-does-maj-and-ch-mean-in-sha-256-algorithm)
   - **Purpose:** Implements a bitwise "choice" function where for each bit position, if the corresponding bit in `x` is `1`, the output takes the bit from `y`; otherwise, it takes the bit from `z`.
   - **Method:** Utilize bitwise operators to combine the bits from `y` and `z` based on `x`.
   - **Application:** This function is commonly used in cryptographic algorithms to blend bits in a controlled manner.

In [72]:
def ch(x, y, z):
    # (x AND y) XOR (NOT x AND z)
    return (x & y) ^ (~ x & z)

# Example
x = 0b1010 # 10
y = 0b0000 # 0  
z = 0b1111 # 15

result = ch(x, y, z)
print("Expeted: 0b101")
print("Result:", bin(result))

Expeted: 0b101
Result: 0b101


## 4. [maj(x, y, z)](https://crypto.stackexchange.com/questions/5358/what-does-maj-and-ch-mean-in-sha-256-algorithm)
   - **Purpose:** Computes the majority vote of the bits in `x`, `y`, and `z` for each bit position. The output bit is `1` if at least two of the three corresponding bits are `1`.
   - **Method:** Combine bitwise ANDs and ORs to efficiently determine the majority.
   - **Application:** This function is another staple in cryptographic hash functions, ensuring robust diffusion of input bits.

In [73]:
def maj(x, y, z):
    # (x AND y) XOR (x AND z) XOR (y AND z)
    return (x & y) ^ (x & z) ^ (y & z)

# Example
x = 0b1010 # 10
y = 0b0000 # 0
z = 0b1111 # 15

print("Expected:", bin(x))
print("Result:  ", bin(maj(x, y, z)))

Expected: 0b1010
Result:   0b1010


[Back to Top](#table-of-contents)

---

# **Task 2: Hash Functions**

In this task, we will convert a classic C hash function from *The C Programming Language* by Brian Kernighan and Dennis Ritchie into Python. The original hash function uses the `ord()` function (in Python, this returns the Unicode code point for a given character) to process each character in a string. For further details on the `ord()` function, see the [ord() Function documentation](https://www.w3schools.com/python/ref_func_ord.asp).

## The Original C Hash Function

The C code for the hash function is as follows:

```c
unsigned hash(char *s) {
    unsigned hashval;
    for (hashval = 0; *s != '\0'; s++)
        hashval = *s + 31 * hashval;
    return hashval % 101;
}


### Explanation:
- **Loop through the string:** The function iterates over each character in the input string `s`.
- **Calculate hash value:** For every character, the hash value is updated by multiplying the current hash value by `31` and then adding the ASCII value of the character. This multiplication factor of ``31`` is chosen because it is a small prime number that helps in spreading out the hash values over a wide range.
- **Modulo Operation:** The final hash value is reduced using a modulo operation with `101`, another prime number, which helps in distributing the hash values more uniformly across available buckets (this can be especially useful for hash tables).

For more details on the concept of hash sizes and the reasoning behind such choices, refer to [ Hash Size in The C Programming Language.](https://www.cimat.mx/ciencia_para_jovenes/bachillerato/libros/%5BKernighan-Ritchie%5DThe_C_Programming_Language.pdf)

In [74]:
def hash(s, base=31, modulus=101):
    hashValue = 0
    for c in s:
        hashValue = ord(c) + base * hashValue # ord(c) returns the ASCII value of c
    return hashValue % modulus 

### Porting to Python
To implement the hash function in Python:

- **Using [`ord()`](https://www.w3schools.com/python/ref_func_ord.asp):** The `ord()` function is used to obtain the integer representation (Unicode code point) of each character in the string.

- **Iteration over the string:** We can iterate over the string directly since Python strings are iterable.

- **Modulo Operation:** The modulo operation `(% 101)` is applied at the end to get the final hash value.

In [75]:
# Some test strings
test_strings = [
    "hello", "world", "foo", "bar", "baz", "hash", "collision", "test",
    "abcd", "abce", "abcf", "java", "python", "rust", "c++", "javascript"
]

for string in test_strings:
    print(f"{string!r} -> {hash(string):>08b}, {hash(string)}")

'hello' -> 00010001, 17
'world' -> 00100010, 34
'foo' -> 01000101, 69
'bar' -> 00100100, 36
'baz' -> 00101100, 44
'hash' -> 00001111, 15
'collision' -> 00001000, 8
'test' -> 01010110, 86
'abcd' -> 01100100, 100
'abce' -> 00000000, 0
'abcf' -> 00000001, 1
'java' -> 01011101, 93
'python' -> 01011011, 91
'rust' -> 00010001, 17
'c++' -> 00111100, 60
'javascript' -> 00101011, 43


*Collison in `rust -> 17` and `hello -> 17`*

## Why `31` and `101`

After testing various candidates—such as bases `[31, 37, 257]` and moduli `[101, 127, 10_007]`—the values `31` and `101` have emerged as effective choices. As highlighted in Joshua Bloch's [*Effective Java*](https://ia800308.us.archive.org/16/items/java_20230528/Joshua%20Bloch%20-%20Effective%20Java%20%283rd%29%20-%202018.pdf):

* The value `31` was chosen because it is an odd prime. If it were even and the multiplication overflowed, information would be lost, because multiplication by `2` is equivalent to shifting. The advantage of using a prime is less clear, but it is traditional.  
* A nice property of `31` is that the multiplication can be replaced by a shift and a subtraction for better performance on some architectures: `31 * i == (i << 5) - i`. Modern VMs do this sort of optimization automatically.
This allows faster computation on certain architectures.
* Since `31 < 32`, it stays within the bit-width of a 32-bit processor.

Using `31` as the multiplier helps to generate a well-distributed hash by combining the contributions of each character in a string. Its odd prime nature avoids the pitfalls of even multipliers—such as losing information on overflow—and the computational shortcut (shifting and subtracting) can improve performance.

Similarly, the modulus `101` is chosen because it is a prime number. A prime modulus helps minimize collisions by ensuring a more uniform distribution of hash values. While the exact advantage of using a prime modulus like `101` might be less pronounced than that of the multiplier, it remains a traditional choice that has been proven effective in practice.

In summary, the combination of `31` and `101` has been historically favored in hash function design due to their mathematical properties and performance benefits, which remain relevant even as testing with other numbers (e.g., `37`, `257` for the base and `127`, `10_007` for the modulus) might show similar behavior in certain scenarios.


[Back to Top](#table-of-contents)

---

# **Task 3: SHA256 Padding**

The task is to implement a Python function that computes the SHA256 padding for a given file. This involves reading the file's contents in binary mode, calculating the padding according to the SHA256 ([SHA-2 - Wikipedia](https://en.wikipedia.org/wiki/SHA-2)) specification as detailed in [NIST FIPS 180-4](https://doi.org/10.6028/NIST.FIPS.180-4), and outputting the resulting padding in hexadecimal format.

In the SHA256 algorithm, before the input message is processed, it must be padded to satisfy the requirement that its total length (in bits) is equal to 448 bytes modulo 512. This is achieved by appending the following to the original message:
- **A single `1` bit:** Represented in hexadecimal as `0x80` (binary: `10000000`).
- **A series of `0` bits:** Enough zeros are appended so that the length of the message (after adding `0x80` and the zeros) is 56 bytes modulo 64. This ensures that, once an 8-byte (64-bit) representation of the original message length is appended, the overall message length is a multiple of 64 bytes (512 bits).
- **The original message length:** The length (in bits) of the original input is appended as a 64-bit big-endian integer.

This process is described in detail in [NIST FIPS 180-4](https://doi.org/10.6028/NIST.FIPS.180-4), which is the official document for the Secure Hash Standard (SHS). Additional context can be found on the [SHA-2 - Wikipedia](https://en.wikipedia.org/wiki/SHA-2) Wikipedia page and in [RFC 6234](https://datatracker.ietf.org/doc/html/rfc6234), which provide overviews and implementation notes for SHA-256.

In summary, the implementation should:
- Open and read the file in binary mode.
- Append `0x80` to the file's contents to indicate the start of the padding.
- Calculate the number of zero bytes needed so that, with the additional 8-byte length field, the total length is a multiple of 64 bytes.
- Append the original message length (in bits) as a 64-bit big-endian integer.


In [ ]:
def sha256_padding(file_path):
    # Read the file contents in binary mode.
    with open(file_path, 'rb') as f:
        data = f.read()
    
    # Compute the original message length in bits.
    original_length = len(data)
    original_bit_length = original_length * 8

    # Step 1: Append the '1' bit as 0x80.
    # This represents the bit pattern 10000000.
    padding = b'\x80'

    # Step 2: Calculate how many zero bytes are needed.
    # After appending 0x80, the message should be padded so that its length (in bytes)
    # is congruent to 56 modulo 64. This leaves room for the final 8-byte length field.
    # That is, we need: (original_length + 1 + pad_len) % 64 == 56.
    pad_len = (56 - (original_length + 1) % 64) % 64
    padding += b'\x00' * pad_len

    # Step 3: Append the original length as a 64-bit big-endian integer.
    padding += struct.pack('>Q', original_bit_length)

    # Return the padding as a hexadecimal string.
    return padding.hex()

# Example
file_path = 'abc.txt'
test_answer = "80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018" # copied from task.md
padding = sha256_padding(file_path)
print("Padding abc.txt: " + padding)
print(padding == test_answer)

Padding abc.txt: 80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018
True


### **How it works**

1. **Reading the file:**
    * file is opened using binary mode (i.e., "rb"). 
    * entire content of the file is read into `data`. 
    * length of the file (in bytes) is stored in a variable (let’s call it `L`)

2. **Appending the `1` Bit:**
    In SHA256 (and other SHA-2 algorithms), the first step of padding is to signal the end of the original message. This is done by appending a single `1` bit to the message.Although only a single bit is needed, bytes are the smallest addressable unit in computer memory. Thus, the 1 bit is appended as part of a full byte. The byte `0x80` in hexadecimal corresponds to the binary value `10000000`. After appending 0x80, the new message consists of the original data followed by this byte.

3. **Zero Padding:**

   We then calculate how many zero bytes are needed so that the total length (original message + `0x80` + zeros) is 56 bytes modulo 64. This ensures that when we later add the 8-byte length, the full message length becomes a multiple of 64 bytes (512 bits).

   * Let `L` be the original length (in bytes).
   * After appending `0x80`, the length becomes `L + 1`.
   * The formula used is:

     ```python
     pad_len = (56 - (L + 1) % 64) % 64
     ```
    This ensures:
    * If `(L + 1)` already leaves a remainder of 56 when divided by 64, no additional zero bytes are required.
    * Otherwise, it computes exactly how many 0x00 bytes are needed to reach the target length of 56 modulo 64.


4. **Appeding the Message Length:**

    The SHA256 padding requires that the last 8 bytes of the padded message represent the length of the original message in bits. This fixed-size representation ensures that the total message length is accurately recorded. Since the original length is in bytes (L), multiplying it by 8 converts it to bits (L * 8). This length is then represented as a 64-bit (8-byte) big-endian integer. In big-endian order, the most significant byte is placed first, which is crucial for consistency across different systems and aligns with the SHA256 specification.

    In Python, this conversion can be done using `struct.pack('>Q', L * 8)`, where '[>Q'](https://docs.python.org/3/library/struct.html) denotes an unsigned 64-bit big-endian integer. Alternatively, you can use `int.to_bytes(8, 'big')` on the computed bit length. This 8-byte representation is appended to the padded message, ensuring that the overall length of the padded message is a multiple of 64 bytes.

[Back to Top](#table-of-contents)

---

# **Task 4: Prime Numbers**
## Introduction

Prime numbers have long been a subject of fascination for mathematicians. A prime number is a natural number greater than 1 that has no positive divisors other than 1 and itself. They form the “building blocks” of integers because every integer can be uniquely factored into primes—a principle known as [the Fundamental Theorem of Arithmetic](https://en.wikipedia.org/wiki/Fundamental_theorem_of_arithmetic). Beyond their theoretical importance, prime numbers are also central to modern cryptography, particularly in secure communication systems like RSA. 

In this task, we will compute the first 100 prime numbers using two different algorithms. We’ll explain how each method works, discuss potential optimizations, and compare their performance.


### [Optimized Trial Division Method - O(√n) Time and O(1) Space](https://www.geeksforgeeks.org/check-for-prime-number/)
Every integer can be written as 6 multiplied by some number `𝑘` plus a remainder `𝑖` (where`𝑖` is 0, 1, 2, 3, 4, or 5). If `𝑖` is 0, 2, 3, or 4, then the number is divisible by 2 or 3. Since primes greater than 3 cannot be divisible by 2 or 3, they must leave a remainder of either 1 or 5 when divided by 6 (i.e. they are of the form `6𝑘+1 or 6k+5 `). This means that when checking if a number is prime, we only need to test numbers in these two forms, which cuts down on the number of checks.

## Algorithm 1: Sieve of Eratosthenes
The Sieve of Eratosthenes is an efficient algorithm to generate all prime numbers up to a given limit. 

### How it works:

1. **Initialization:** Creating a boolean list where each index represents a number. Initially, all numbers are assumed to be prime (set to True), except for 0 and 1.
2. **Sieving Process:** Starting from the first prime (2), the algorithm marks all multiples of each prime as non-prime (False). The process continues with the next number that is still marked as prime.
3. **Collection:** After processing up to the square root of the limit, the remaining indices marked as True correspond to prime numbers.

Since the 100th prime is 541, choosing a limit of 600 ensures that we capture at least 100 primes.

For more details on the algorithm, you can refer [the Sieve of Eratosthenes.](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes)

In [ ]:
def sieve_of_eratosthenes(limit):
    # Create a list of boolean values, initially set to True.
    is_prime = [True] * (limit + 1)
    # 0 and 1 are not prime numbers.
    is_prime[0] = is_prime[1] = False

    # Iterate from 2 up to the square root of the limit.
    for i in range(2, int(limit ** 0.5) + 1):
        if is_prime[i]:
            # Mark multiples of i as non-prime.
            for j in range(i * i, limit + 1, i):
                is_prime[j] = False

    # Return the list of numbers that are prime.
    return [num for num, prime in enumerate(is_prime) if prime]

# Since the 100th prime is 541, we set the limit to 600 to ensure we have at least 100 primes.
primes = sieve_of_eratosthenes(542)

print("The first", len(primes), "prime numbers are (Sieve of Eratosthenes):")
print(primes)

The first 100 prime numbers are (Sieve of Eratosthenes):
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


## Algorithm 2: Sieve of Sundaram
The Sieve of Sundaram is a prime number sieve that generates all prime numbers less than 2n + 2.

### How it works:
1. Create a list of integers from 1 to n (where n is an upper bound derived from the required number of primes).
2. Eliminate numbers of the form `i + j + 2ij` where `1 <= i <= j` and `i + j + 2ij <= n`.
3. The remaining numbers, when transformed into `2x + 1`, represent the prime numbers.

The advantage of this sieve is that it eliminates non-primes efficiently without requiring division

In [ ]:
def sieve_of_sundaram(limit):
    n = (limit - 1) // 2  # We are interested in primes < 2n + 2
    sieve = [True] * (n + 1)
    
    for i in range(1, n + 1):
        for j in range(i, (n - i) // (2 * i + 1) + 1):
            sieve[i + j + 2 * i * j] = False  # Mark composite numbers
    
    primes = [2] + [2 * x + 1 for x in range(1, n + 1) if sieve[x]]  # Convert indices to primes
    return primes[:limit]

# Since the 100th prime is 541, we set the limit to 542 to ensure we have at least 100 primes.
first_100_primes = sieve_of_sundaram(542)

print("The first", len(first_100_primes), "prime numbers are (Sieve of Sundaram):")
print(first_100_primes)

The first 100 prime numbers are (Sieve of Sundaram):
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


## Comparison of Prime Number Sieves

Various algorithms have been developed to efficiently generate prime numbers. Three notable sieves are:

1. [**Sieve of Eratosthenes**](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes): An ancient algorithm that iteratively marks the multiples of each prime number starting from 2. The numbers that remain unmarked are prime.

2. [**Sieve of Sundaram**](https://en.wikipedia.org/wiki/Sieve_of_Sundaram): A sieve that eliminates numbers of the form `i + j + 2ij` (where `1 ≤ i ≤ j`) from a list of integers, then transforms the remaining numbers to obtain primes.

3. [**Sieve of Atkin**](https://www.geeksforgeeks.org/sieve-of-atkin/): A modern algorithm that uses quadratic forms and modular arithmetic to identify primes, aiming for improved efficiency over the Sieve of Eratosthenes.

### Performance Comparison

| Sieve Algorithm           | Time Complexity       | Space Complexity | Pros                                         | Cons                                               |
|---------------------------|-----------------------|------------------|----------------------------------------------|----------------------------------------------------|
| **Sieve of Eratosthenes** | $O(n \log \log n)$  | $O(n)$         | Simple implementation; efficient for moderate ranges. | Less efficient for very large numbers due to memory constraints. |
| **Sieve of Sundaram**     | $O(n \log n)$       | $O(n)$         | Operates only on odd numbers, reducing space usage. | Requires additional computation to generate the final list of primes. |
| **Sieve of Atkin**        | $O(n / \log \log n)$ | $O(n)$         | More efficient for generating large primes.  | Complex implementation; less efficient for smaller ranges. |


### Detailed Analysis

- **Sieve of Eratosthenes**: This algorithm is straightforward and effective for generating all primes up to a moderate limit. Its simplicity makes it a common choice for many applications. However, its performance can degrade for very large numbers due to increased memory usage.

- **Sieve of Sundaram**: By focusing solely on odd numbers, this sieve reduces the number of operations and space required. After eliminating numbers of the form `i + j + 2ij`, the remaining numbers are transformed to obtain the primes. While it offers some efficiency benefits, it involves extra steps to produce the final list of primes.

- **Sieve of Atkin**: Designed to be more efficient for large ranges, this algorithm uses mathematical properties to identify potential primes and then filters out composites. Its theoretical efficiency is higher, but the complexity of implementation and operations can make it less practical for smaller ranges.

### Conclusion

- **For small to medium-sized prime lists**, the **Sieve of Eratosthenes** is often preferred due to its simplicity and adequate performance.

- **In memory-constrained environments**, the **Sieve of Sundaram** offers a space-efficient alternative by operating only on odd numbers.

- **For generating primes in very large ranges**, the **Sieve of Atkin** provides better theoretical performance, though its complexity may outweigh benefits for smaller applications.

### Additional References

- [Harahap, M. K., & Khairina, N. (2019). *The Comparison of Methods for Generating Prime Numbers between The Sieve of Eratosthenes, Atkins, and Sundaram*. SinkrOn, 3(2), 293-298. ](https://jurnal.polgan.ac.id/index.php/sinkron/article/view/10129)

- ["Eratosthenes/Sundaram/Atkins Sieve Implementation in C." CodeProject. ](https://www.codeproject.com/Articles/490085/Eratosthenes-Sundaram-Atkins-Sieve-Implementation)


[Back to Top](#table-of-contents)

---

# **Task 5: Roots**
**Calculate the first 32 bits of the fractional part of the square roots of the first 100 prime numbers.**

We can show that for any prime number ***p*** the “first 32 bits of the fractional part of its square root” is defined by $$\text{value}(p) = \left\lfloor \left(\sqrt{p} - \lfloor \sqrt{p} \rfloor\right) \times 2^{32} \right\rfloor.$$

In other words, if you write $$\sqrt{p} = N.f_1f_2f_3\cdots \quad (\text{in binary}),$$
then the number $$\left\lfloor \left(\sqrt{p} - N\right) \times 2^{32} \right\rfloor$$

is exactly the 32‐bit unsigned integer whose bit–string is the “first 32 bits” of the fractional part.

For example, for $$p = 2$$ we have

$$
\sqrt{2} \approx 1.4142135623730950 \quad \Rightarrow \quad \sqrt{2} - 1 \approx 0.4142135623730950,
$$

and

$$
0.4142135623730950 \times 2^{32} \approx 1779033703.
$$

In hexadecimal this number is

$$
1779033703 \;=\; 0x6a09e667.
$$

In fact, it turns out that the values for the first eight primes are exactly the same numbers used as the “initial hash values” in SHA‑256:

| Prime $$p$$ | $$\sqrt{p}$$ (approx.)     | Fractional part $$\sqrt{p} - \lfloor \sqrt{p} \rfloor$$ | $$\left\lfloor \left(\sqrt{p} - \lfloor \sqrt{p} \rfloor\right) \times 2^{32} \right\rfloor$$ (hex) |
|:-----------:|:--------------------------:|:---------------------------------------------------------:|:--------------------------------------------------------------------------------------------:|
| 2           | 1.414213562373095          | 0.414213562373095                                         | 0x6a09e667                                                                                   |
| 3           | 1.732050807568877          | 0.732050807568877                                         | 0xbb67ae85                                                                                   |
| 5           | 2.236067977499790          | 0.236067977499790                                         | 0x3c6ef372                                                                                   |
| 7           | 2.645751311064591          | 0.645751311064591                                         | 0xa54ff53a                                                                                   |
| 11          | 3.316624790355400          | 0.316624790355400                                         | 0x510e527f                                                                                   |
| 13          | 3.605551275463990          | 0.605551275463990                                         | 0x9b05688c                                                                                   |
| 17          | 4.123105625617661          | 0.123105625617661                                         | 0x1f83d9ab                                                                                   |
| 19          | 4.358898943               | 0.358898943                                              | 0x5be0cd19                                                                                   |

To perform the calculation for the first 100 primes we can use the Sieve of Eratosthenes `sieve_of_eratosthenes(limit)`.

For example, the following code computes the desired 32‐bit numbers (shown here in hexadecimal):

In [ ]:
def first_32_frac_bits(n):
    # Compute the fractional part of sqrt(n) and scale it to 32 bits
    frac = math.sqrt(n) - math.floor(math.sqrt(n))
    return int(frac * 2**32)

# Since the 100th prime is 541, we set the limit to 600 to ensure we have at least 100 primes.
primes = sieve_of_eratosthenes(600)
results = {p: hex(first_32_frac_bits(p)) for p in primes}
# Print the results in a neat table
for p in primes:
    print(f"{p:3}: {results[p]}")

  2: 0x6a09e667
  3: 0xbb67ae85
  5: 0x3c6ef372
  7: 0xa54ff53a
 11: 0x510e527f
 13: 0x9b05688c
 17: 0x1f83d9ab
 19: 0x5be0cd19
 23: 0xcbbb9d5d
 29: 0x629a292a
 31: 0x9159015a
 37: 0x152fecd8
 41: 0x67332667
 43: 0x8eb44a87
 47: 0xdb0c2e0d
 53: 0x47b5481d
 59: 0xae5f9156
 61: 0xcf6c85d3
 67: 0x2f73477d
 71: 0x6d1826ca
 73: 0x8b43d457
 79: 0xe360b596
 83: 0x1c456002
 89: 0x6f196331
 97: 0xd94ebeb1
101: 0xcc4a611
103: 0x261dc1f2
107: 0x5815a7be
109: 0x70b7ed67
113: 0xa1513c69
127: 0x44f93635
131: 0x720dcdfd
137: 0xb467369e
139: 0xca320b75
149: 0x34e0d42e
151: 0x49c7d9bd
157: 0x87abb9f2
163: 0xc463a2fc
167: 0xec3fc3f3
173: 0x27277f6d
179: 0x610bebf2
181: 0x7420b49e
191: 0xd1fd8a33
193: 0xe4773594
197: 0x92197f6
199: 0x1b530c95
211: 0x869d6342
223: 0xeee52e4f
227: 0x11076689
229: 0x21fba37b
233: 0x43ab9fb6
239: 0x75a9f91d
241: 0x86305019
251: 0xd7cd8173
257: 0x7fe00ff
263: 0x379f513f
269: 0x66b651a8
271: 0x764ab842
277: 0xa4b06be1
281: 0xc3578c15
283: 0xd2962a53
293: 0x1e039f40
307: 0x857b

[Back to Top](#table-of-contents)

---

## **Task 6: Proof of Work**
...

[Back to Top](#table-of-contents)

---

## **Task 7: Turing Machines**
...

[Back to Top](#table-of-contents)

---

## **Task 8: Computational Complexity**
...

[Back to Top](#table-of-contents)

---

## **Conclusion**
...

[Back to Top](#table-of-contents)

---